<a href="https://colab.research.google.com/github/yananma/5_programs_per_day/blob/master/pytorch/ch3/3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3.3 线性回归的简洁实现

### 3.3.1 生成数据集

In [0]:
import torch 
from torch import nn 
import numpy as np 

In [0]:
num_inputs = 2 
num_examples = 1000 
true_w = [2, -3.4]
true_b = 4.2 
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b 
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

### 3.3.2 读取数据集

In [0]:
import torch.utils.data as Data 

batch_size = 10 
dataset = Data.TensorDataset(features, labels)
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [0]:
# Data.DataLoader??
# Data loader. Combines a dataset and a sampler, and provides an iterable over the given dataset.

In [7]:
for X, y in data_iter:
    print(X, '\n', y)
    break 

tensor([[ 1.4424, -0.4708],
        [ 0.8540, -0.9407],
        [ 1.6333, -0.6412],
        [ 0.9610,  0.7797],
        [-0.0249,  1.3809],
        [ 0.4529, -1.2042],
        [-1.3523, -2.3790],
        [ 2.1326,  0.7625],
        [ 0.4740, -1.8160],
        [ 2.6771, -0.1584]]) 
 tensor([ 8.6925,  9.1228,  9.6347,  3.4623, -0.5443,  9.1931,  9.5950,  5.8613,
        11.3102, 10.0727])


### 3.3.3 定义模型

In [9]:
# 内部实现，和这一节无关
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)  # 阿拉伯数字 1，输出 1 个

    def forward(self, x):
        y = self.linear(x)
        return y 

net = LinearNet(num_inputs)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [10]:
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
)

print(net)
print(net[0])

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [12]:
# 查看模型所有的可学习参数，也就是 weight 和 bias
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.4866,  0.1314]], requires_grad=True)
Parameter containing:
tensor([-0.2363], requires_grad=True)


### 3.3.4 初始化模型参数

In [13]:
from torch.nn import init 

init.normal_(net[0].weight, mean=0.0, std=0.01)
init.constant_(net[0].bias, val=0.0)

Parameter containing:
tensor([0.], requires_grad=True)

### 3.3.5 定义损失函数

In [0]:
loss = nn.MSELoss()

In [17]:
# my_y_hat = torch.tensor([0., 0., 0.])
# my_y = torch.tensor([3., 3., 3.])
# loss(my_y_hat, my_y)  

# 对应相减
# 每一项平方
# 所有项求和
# 最后求平均

tensor(9.)

### 3.3.6 定义优化算法

In [18]:
from torch import optim 

optimizer = optim.SGD(net.parameters(), lr=0.03)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [21]:
num_epochs = 3 
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' %(epoch, l.item()))

epoch 1, loss: 0.000753
epoch 2, loss: 0.000074
epoch 3, loss: 0.000043


In [0]:
# optimizer.zero_grad??

# Clears the gradients of all optimized :class:`torch.Tensor 

In [0]:
# l.backward??

# Computes the gradient of current tensor w.r.t. graph leaves.

# 使用前，要清零
# This function accumulates gradients in the leaves - you might need to zero them before calling it.

In [0]:
# optimizer.step??

# Performs a single optimization step.

In [23]:
true_w, net[0].weight.data

([2, -3.4], tensor([[ 1.9994, -3.4002]]))

In [24]:
true_b, net[0].bias.data

(4.2, tensor([4.2003]))